<h2> Load Data</h2>

In [1]:
import os
import warnings

import ipynb.fs.full.GenSnippetsLib as snippet
import matplotlib.image as mpimg

%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import preparers
import seaborn as sns
import sonaion_analysis as son
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
data = preparers.load_queried(participant_number=1, snippets=["BogoSort", "Ackerman"], query_code=True, query_eeg=True, query_eye_tracking=True, query_behavioral=True, query_log=True, query_meta=True,)

Loading Ackerman Files ......
Loading BogoSort Files ......


<h2>potential funtion arguments</h2>

In [3]:
fig, ax = (None, None)
denoise_degree = 10
display_width = 1920
display_height = 1080
left_color = (1.0, 0.0, 0.0)
right_color = (0.0, 1.0, 0.0)
image_path = "../CodeSnippets/Generators/" + "BogoSort" + ".json"
img, result = snippet.create_image(image_path, font_path="/../CodeSnippets/fonts/ttf/")
bg = snippet.create_background(display_width, display_height, (180, 180, 180, 255))
img = snippet.place_image_on(bg, np.array(img), 0.5, 0.5)

<h2> Prepare queried Data</h2>

In [4]:
df_eyetracking = data["BogoSort"]["Code"]["EyeTracking"][["l_valid", "r_valid", "l_display_x", "l_display_y", "r_display_x", "r_display_y", "time", "l_pupil_diameter", "r_pupil_diameter",]]
df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].apply(lambda x: x * display_width)
df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].astype(int)

df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].apply(lambda x: x * display_width)
df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].astype(int)

df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].apply(lambda x: x * display_height)
df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].astype(int)

df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].apply(lambda x: x * display_height)
df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].astype(int)

df_eyetracking["time"] = df_eyetracking["time"] - df_eyetracking["time"][0]

In [5]:
df_eyetracking

,l_valid,r_valid,l_display_x,l_display_y,r_display_x,r_display_y,time,l_pupil_diameter,r_pupil_diameter
0,1,1,831,328,833,317,0.000000,3.526718,3.530930
1,1,1,825,317,828,309,0.004000,3.510849,3.528107
2,1,1,829,320,830,307,0.008000,3.513443,3.512985
3,1,1,836,321,834,312,0.012001,3.510406,3.534103
4,1,1,822,312,830,299,0.016000,3.490219,3.501907
...,...,...,...,...,...,...,...,...,...
4260,1,1,949,554,930,557,17.040607,3.313324,3.368393
4261,1,1,934,549,919,557,17.044607,3.319641,3.379089
4262,1,1,936,560,930,565,17.048607,3.348572,3.392838
4263,1,1,927,553,928,550,17.052607,3.316315,3.375214


In [6]:
replaced_r_x, replaced_r_y, replaced_r_diameter, replaced_r_valid = son.eyetracking.preprocessing_invalid.replace_with_prev_invalid(
    df_eyetracking["r_display_x"], df_eyetracking["r_display_y"], df_eyetracking["r_pupil_diameter"], df_eyetracking["r_valid"]
)
df_eyetracking["r_display_x"] = replaced_r_x
df_eyetracking["r_display_y"] = replaced_r_y
df_eyetracking["r_pupil_diameter"] = replaced_r_diameter
df_eyetracking["r_valid"] = replaced_r_valid

replaced_l_x, replaced_l_y, replaced_l_diameter, replaced_l_valid = son.eyetracking.preprocessing_invalid.replace_with_prev_invalid(
    df_eyetracking["l_display_x"], df_eyetracking["l_display_y"], df_eyetracking["l_pupil_diameter"], df_eyetracking["l_valid"]
)

df_eyetracking["l_display_x"] = replaced_l_x
df_eyetracking["l_display_y"] = replaced_l_y
df_eyetracking["l_pupil_diameter"] = replaced_l_diameter
df_eyetracking["l_valid"] = replaced_l_valid

In [7]:
for _i in range(denoise_degree):
    denoised_r_x, denoised_r_y = son.eyetracking.preprocessing_denoise.denoise(df_eyetracking["r_display_x"], df_eyetracking["r_display_y"], son.eyetracking.preprocessing_denoise.parabola_5_kernel)

    denoised_l_x, denoised_l_y = son.eyetracking.preprocessing_denoise.denoise(df_eyetracking["l_display_x"], df_eyetracking["l_display_y"], son.eyetracking.preprocessing_denoise.parabola_5_kernel)

    df_eyetracking["r_display_x"] = denoised_r_x
    df_eyetracking["r_display_y"] = denoised_r_y

    df_eyetracking["l_display_x"] = denoised_l_x
    df_eyetracking["l_display_y"] = denoised_l_y

<h2> Make Heatmaps</h2>

In [8]:
def mask_function():
    return lambda height, width, coordinate: son.utils.masks.create_circular_mask(height, width, coordinate, 30)


data_left = son.eyetracking.heatmap.create_heatmap(df_eyetracking["l_display_x"], df_eyetracking["l_display_y"], df_eyetracking["l_valid"], 0.004, display_width, display_height, mask_function())
data_right = son.eyetracking.heatmap.create_heatmap(df_eyetracking["r_display_x"], df_eyetracking["r_display_y"], df_eyetracking["r_valid"], 0.004, display_width, display_height, mask_function())

data_left = data_left / data_left.max()
data_right = data_right / data_right.max()

data_right = np.array([data_right * right_color[0], data_right * right_color[1], data_right * right_color[2], data_right])
data_right = np.moveaxis(data_right, 0, 2)

data_left = np.array([data_left * left_color[0], data_left * left_color[1], data_left * left_color[2], data_left])
data_left = np.moveaxis(data_left, 0, 2)

In [9]:
fig, ax = plt.subplots()
ax.imshow(data_right, zorder=2, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()

In [10]:
fig, ax = plt.subplots()
ax.imshow(data_left, zorder=2, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()

In [11]:
fig, ax = plt.subplots()
ax.imshow(data_left, zorder=2, alpha=0.5)
ax.imshow(data_right, zorder=3, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()

<h2>Make Sequence Diagrams</h2>

In [12]:
data_right = son.eyetracking.sequence.create_sequence_diagram_y(df_eyetracking["r_display_y"], df_eyetracking["r_valid"], display_height, display_width, offset=670, step=0.1, should_skip=True)
data_right = np.array([data_right * right_color[0], data_right * right_color[1], data_right * right_color[2], data_right])
data_right = np.moveaxis(data_right, 0, 2)

data_left = son.eyetracking.sequence.create_sequence_diagram_y(df_eyetracking["l_display_y"], df_eyetracking["l_valid"], display_height, display_width, offset=670, step=0.1, should_skip=False)
data_left = np.array([data_left * left_color[0], data_left * left_color[1], data_left * left_color[2], data_left])
data_left = np.moveaxis(data_left, 0, 2)

In [13]:
fig, ax = plt.subplots()
ax.imshow(data_left, zorder=2, alpha=0.5)
ax.imshow(data_right, zorder=3, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()

<h2> Make marker</h2>

In [14]:
nmb = 0
data_right = son.eyetracking.field_of_view.create_field_of_view_image(df_eyetracking["r_display_x"][nmb], df_eyetracking["r_display_y"][nmb], display_height, display_width, 32, 30, son.utils.masks.create_circular_mask)
data_right = np.array([data_right * right_color[0], data_right * right_color[1], data_right * right_color[2], data_right])
data_right = np.moveaxis(data_right, 0, 2)

data_left = son.eyetracking.field_of_view.create_field_of_view_image(df_eyetracking["r_display_x"][nmb], df_eyetracking["r_display_y"][nmb], display_height, display_width, 32, 30, son.utils.masks.create_circular_mask)
data_left = np.array([data_left * left_color[0], data_left * left_color[1], data_left * left_color[2], data_left])
data_left = np.moveaxis(data_left, 0, 2)

In [15]:
fig, ax = plt.subplots()
ax.imshow(data_left, zorder=2, alpha=0.5)
ax.imshow(data_right, zorder=3, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()

<h2>Fixxations and Saccades</h2>

In [16]:
treshhold = 2000.0
d_time = 0.004

right_saccades_color = (1.0, 0.0, 1.0)
right_fixxation_color = (0.0, 1.0, 0.0)
saccades_r = son.eyetracking.metrics.classify_saccades(df_eyetracking["r_display_x"], df_eyetracking["r_display_y"], d_time, treshhold)
saccades_r[0] = saccades_r[1]
fixxation_r = son.eyetracking.metrics.classify_fixxation(df_eyetracking["r_display_x"], df_eyetracking["r_display_y"], d_time, treshhold)
fixxation_r[0] = fixxation_r[1]

saccades_r = np.array(saccades_r)
fixxation_r = np.array(fixxation_r)
classificator_right = np.add(saccades_r, np.multiply(fixxation_r, 2))


left_saccades_color = (0.0, 1.0, 1.0)
left_fixxation_color = (1.0, 0.0, 0.0)
saccades_l = son.eyetracking.metrics.classify_saccades(df_eyetracking["l_display_x"], df_eyetracking["l_display_y"], d_time, treshhold)
saccades_l[0] = saccades_l[1]
fixxation_l = son.eyetracking.metrics.classify_fixxation(df_eyetracking["l_display_x"], df_eyetracking["l_display_y"], d_time, treshhold)
fixxation_l[0] = fixxation_l[1]

saccades_l = np.array(saccades_l)
fixxation_l = np.array(fixxation_l)
classificator_left = np.add(saccades_l, np.multiply(fixxation_l, 2))

<h2> Create Fixxation Map</h2>

In [17]:
data_right = son.eyetracking.sequence.create_classified_sequence_diagram_y(df_eyetracking["r_display_y"], classificator_right, display_height, display_width, offset=10, step=0.1)

data_right_saccades = np.array(data_right).astype(int)
data_right_saccades[np.where(data_right_saccades == 2)] = 0
data_right_saccades = np.array([data_right_saccades * right_saccades_color[0], data_right_saccades * right_saccades_color[1], data_right_saccades * right_saccades_color[2], data_right_saccades])
data_right_saccades = np.moveaxis(data_right_saccades, 0, 2)

data_right_fixxations = np.array(data_right).astype(int)
data_right_fixxations[np.where(data_right_fixxations == 1)] = 0
data_right_fixxations = data_right_fixxations / 2
data_right_fixxations = np.array([data_right_fixxations * right_fixxation_color[0], data_right_fixxations * right_fixxation_color[1], data_right_fixxations * right_fixxation_color[2], data_right_fixxations])
data_right_fixxations = np.moveaxis(data_right_fixxations, 0, 2)


data_left = son.eyetracking.sequence.create_classified_sequence_diagram_y(df_eyetracking["l_display_y"], classificator_left, display_height, display_width, offset=10, step=0.1)

data_left_saccades = np.array(data_left).astype(int)
data_left_saccades[np.where(data_left_saccades == 2)] = 0
data_left_saccades = np.array([data_left_saccades * left_saccades_color[0], data_left_saccades * left_saccades_color[1], data_left_saccades * left_saccades_color[2], data_left_saccades])
data_left_saccades = np.moveaxis(data_left_saccades, 0, 2)

data_left_fixxations = np.array(data_left).astype(int)
data_left_fixxations[np.where(data_left_fixxations == 1)] = 0
data_left_fixxations = data_left_fixxations / 2
data_left_fixxations = np.array([data_left_fixxations * left_fixxation_color[0], data_left_fixxations * left_fixxation_color[1], data_left_fixxations * left_fixxation_color[2], data_left_fixxations])
data_left_fixxations = np.moveaxis(data_left_fixxations, 0, 2)

In [18]:
fig, ax = plt.subplots()
ax.imshow(data_right_fixxations, zorder=2, alpha=0.8)
ax.imshow(data_right_saccades, zorder=2, alpha=0.8)
ax.imshow(img, zorder=1)
plt.show()

In [19]:
fig, ax = plt.subplots()
ax.imshow(data_left_fixxations, zorder=2, alpha=0.8)
ax.imshow(data_left_saccades, zorder=2, alpha=0.8)
ax.imshow(img, zorder=1)
plt.show()

In [20]:
fig, ax = plt.subplots()
ax.imshow(data_right_fixxations, zorder=2, alpha=0.8)
ax.imshow(data_right_saccades, zorder=2, alpha=0.8)
ax.imshow(data_left_fixxations, zorder=2, alpha=0.8)
ax.imshow(data_left_saccades, zorder=2, alpha=0.8)
ax.imshow(img, zorder=1)
plt.show()

In [21]:
print("Left Metrics")
print("Saccades: Saccedes Count: ", son.eyetracking.metrics.count_saccades(saccades_l))
print("Saccades: Time to first Saccade: ", son.eyetracking.metrics.time_of_saccades(saccades_l, d_time)[0][0], "s")
print("Saccades: Average Saccade Time: ", son.eyetracking.metrics.average_saccades_time(son.eyetracking.metrics.time_of_saccades(saccades_l, d_time)))
print("Fixxation: Fixxation Count: ", son.eyetracking.metrics.count_saccades(fixxation_l))
print("Fixxation: Time to first Fixxation: ", son.eyetracking.metrics.time_of_saccades(fixxation_l, d_time)[0][0], "s")
print("Fixxation: Average Fixxation Time: ", son.eyetracking.metrics.average_saccades_time(son.eyetracking.metrics.time_of_saccades(fixxation_l, d_time)))

Left Metrics
Saccades: Saccedes Count:  38
Saccades: Time to first Saccade:  1.0640000000000007 s
Saccades: Average Saccade Time:  0.024210526315788582
Fixxation: Fixxation Count:  39
Fixxation: Time to first Fixxation:  0.0 s
Fixxation: Average Fixxation Time:  0.41999999999997395


In [22]:
print("Right Metrics")
print("Saccades: Saccedes Count: ", son.eyetracking.metrics.count_saccades(saccades_r))
print("Saccades: Time to first Saccade: ", son.eyetracking.metrics.time_of_saccades(saccades_r, d_time)[0][0], "s")
print("Saccades: Average Saccade Time: ", son.eyetracking.metrics.average_saccades_time(son.eyetracking.metrics.time_of_saccades(saccades_r, d_time)))
print("Fixxation: Fixxation Count: ", son.eyetracking.metrics.count_saccades(fixxation_r))
print("Fixxation: Time to first Fixxation: ", son.eyetracking.metrics.time_of_saccades(fixxation_r, d_time)[0][0], "s")
print("Fixxation: Average Fixxation Time: ", son.eyetracking.metrics.average_saccades_time(son.eyetracking.metrics.time_of_saccades(fixxation_r, d_time)))

Right Metrics
Saccades: Saccedes Count:  39
Saccades: Time to first Saccade:  1.0640000000000007 s
Saccades: Average Saccade Time:  0.021641025641024828
Fixxation: Fixxation Count:  40
Fixxation: Time to first Fixxation:  0.0 s
Fixxation: Average Fixxation Time:  0.41117948717946173


<h2>Fixxation Map</h2>

In [23]:
from sonaion_analysis.utils.smallestenclosingcircle import make_circle


def create_fixxation_map(eye_x, eye_y, fixxation_classifier):
    points_array = []
    currently_fixxation = False
    current_points = []

    for idx, classifier in enumerate(fixxation_classifier):
        if classifier == 1 and currently_fixxation == False:
            current_points = [(eye_x[idx], eye_y[idx])]
            currently_fixxation = True
        elif classifier == 1:
            current_points.append((eye_x[idx], eye_y[idx]))
        elif classifier == 0 and currently_fixxation == True:
            points_array.append((current_points.copy(), True))
            current_points = []
            currently_fixxation = False
            points_array.append(([(eye_x[idx], eye_y[idx])], False))
        else:
            points_array.append(([(eye_x[idx], eye_y[idx])], False))

    circles = [(make_circle(points), is_fixxation) for points, is_fixxation in points_array]
    circles = [((x, y), radius, is_fixxation) for ((x, y, radius), is_fixxation) in circles]

    return circles


def interpolate(start_color, end_color, current, max_value):
    t = current / float(max_value)
    r = (1.0 - t) * start_color[0] + t * end_color[0]
    g = (1.0 - t) * start_color[1] + t * end_color[1]
    b = (1.0 - t) * start_color[2] + t * end_color[2]
    return (r, g, b)

In [24]:
fixxation_map_l = create_fixxation_map(df_eyetracking["l_display_x"], df_eyetracking["l_display_y"], fixxation_l)
fixxation_map_r = create_fixxation_map(df_eyetracking["r_display_x"], df_eyetracking["r_display_y"], fixxation_r)

In [25]:
fig, ax = plt.subplots()
x_val = [float(int(x)) for ((x, _), _, _) in fixxation_map_l]
y_val = [float(int(y)) for ((_, y), _, _) in fixxation_map_l]

for i in range(len(x_val) - 2):
    ax.plot(x_val[i : i + 2], y_val[i : i + 2], "-", color=interpolate(left_color, (0.0, 0.0, 0.0), i, len(x_val)), zorder=2, alpha=0.6)

filtered_x_val = [float(int(x)) for ((x, _), _, flag) in fixxation_map_l if flag]
filtered_y_val = [float(int(y)) for ((_, y), _, flag) in fixxation_map_l if flag]
ax.scatter(filtered_x_val, filtered_y_val, 30, c=[interpolate(left_color, (0.0, 0.0, 0.0), idx, len(x_val)) for idx, (_, _, flag) in enumerate(fixxation_map_l) if flag], zorder=3, alpha=0.6)
ax.imshow(img, zorder=1)
plt.show()

In [26]:
fig, ax = plt.subplots()
x_val = [float(int(x)) for ((x, _), _, _) in fixxation_map_r]
y_val = [float(int(y)) for ((_, y), _, _) in fixxation_map_r]

for i in range(len(x_val) - 2):
    ax.plot(x_val[i : i + 2], y_val[i : i + 2], "-", color=interpolate(right_color, (0.0, 0.0, 0.0), i, len(x_val)), zorder=2, alpha=0.6)

filtered_x_val = [float(int(x)) for ((x, _), _, flag) in fixxation_map_r if flag]
filtered_y_val = [float(int(y)) for ((_, y), _, flag) in fixxation_map_r if flag]
ax.scatter(filtered_x_val, filtered_y_val, 30, c=[interpolate(right_color, (0.0, 0.0, 0.0), idx, len(x_val)) for idx, (_, _, flag) in enumerate(fixxation_map_r) if flag], zorder=3, alpha=0.6)
ax.imshow(img, zorder=1)
plt.show()